# Laboratorio 6 - Parte 2

## Integrantes

- Sergio Orellana 221122
- Rodrigo Mansilla 22611
- Carlos Valladares 221164

# Parte 2 - Teoría

### Pregunta 1
**La matriz Jacobiana es una "instantánea" de las fuerzas del sistema en un punto específico. Explique conceptualmente qué representa el elemento en la esquina inferior izquierda de la matriz $\left(\frac{\partial (dI/dt)}{\partial S}\right)$. ¿Qué nos dice este término sobre la relación entre los susceptibles y la propagación de la infección?**

**Respuesta:**  
Para el SIR con demografía, $\displaystyle \frac{dI}{dt} = \beta \frac{S I}{N} - (\gamma+\mu) I$. Su derivada parcial respecto a $S$ es
$$
\frac{\partial}{\partial S}\!\left(\frac{dI}{dt}\right) = \beta \frac{I}{N}.
$$
Conceptualmente, mide cuán sensible es la tasa de cambio de infectados a un pequeño aumento de susceptibles en ese punto del estado. Indica cuánto “combustible” adicional (susceptibles) acelera el crecimiento de $I$ dado el $I$ actual.


### Pregunta 2
**El análisis muestra que cuando $R_0>1$, el Equilibrio Libre de Enfermedad es inestable. Describa en una o dos frases qué significa esto en el contexto de una política de salud pública de "esperar y ver". ¿Por qué la inestabilidad de este punto hace que esa política sea arriesgada?**

**Respuesta:**  
Significa que cualquier introducción pequeña de infectados tiende a crecer en lugar de extinguirse: las trayectorias cerca del ELE se alejan de ese punto. En términos de política pública, “esperar y ver” es arriesgado porque, sin intervención, los casos aumentarán (a menudo inicialmente de forma exponencial), complicando el control posterior.


### Pregunta 3
**El Equilibrio Endémico, cuando existe, suele ser estable. ¿Qué significa "estabilidad" en este contexto? Si una nueva variante del virus, más contagiosa, causa un aumento repentino de casos (una perturbación), ¿qué predice la estabilidad de este equilibrio sobre el comportamiento del sistema a largo plazo?**

**Respuesta:**  
Estabilidad significa que perturbaciones pequeñas hacen que el sistema retorne hacia un equilibrio. Si surge una variante más contagiosa (mayor $\beta$), el sistema tenderá a estabilizarse alrededor de un nuevo equilibrio con mayor $I^*$ y menor $S^*$ (porque $S^* = N/R_0$ disminuye cuando $R_0$ aumenta), típicamente mediante oscilaciones amortiguadas hasta asentarse.


### Pregunta 4
**En un modelo de ecuaciones, un equilibrio estable es un punto fijo. En un MBA, debido a la aleatoriedad (estocasticidad), el sistema nunca se asienta perfectamente en un punto. ¿Cómo se manifestaría un "equilibrio estable" en el espacio de estados proyectado de un MBA? Describa cómo se vería la trayectoria del sistema una vez que ha alcanzado este estado de equilibrio.**

**Respuesta:**  
En un MBA no se alcanza un punto fijo exacto por la aleatoriedad. En cambio, el sistema entra en un régimen estacionario donde la trayectoria de $(S(t), I(t))$ merodea alrededor del punto (o región) de equilibrio: se observa una nube compacta o ciclo difuso con fluctuaciones alrededor de valores promedio estables. La distribución de los puntos en esa nube se mantiene aproximadamente constante en el tiempo (estacionariedad), aunque cada realización individual siga variando.

# Práctica

## Modelo Simbólico y calculo de Jacobiano

Se definen las ecuaciones:

$$
\begin{align*}
\frac{dS}{dt} = \mu N - \beta S I - \mu S \\
\frac{dI}{dt} = \beta S I - \gamma I - \mu I
\end{align*}
$$

A partir de estas expresiones se construye el **Jacobiano**, una matriz que describe cómo pequeños cambios en $S$ e $I$ afectan la dinámica del sistemas.

In [2]:
import sympy as sp
import numpy as np

S, I, beta, gamma, mu, N = sp.symbols('S I beta gamma mu N', positive=True)

dSdt = mu*N - beta*S*I - mu*S
dIdt = beta*S*I - gamma*I - mu*I

eqs = sp.Matrix([dSdt, dIdt])
vars = sp.Matrix([S, I])

J = eqs.jacobian(vars)
print("Matriz Jacobiana simbólica:")
display(J)


Matriz Jacobiana simbólica:


Matrix([
[-I*beta - mu,             -S*beta],
[      I*beta, S*beta - gamma - mu]])

## Autovalores simbólicos en el Equilibrio Libre de Enfermedad


Se sustituye este punto en el Jacobiano simbólico y se calculan sus autovalores.  Estos autovalores indican si pequeñas perturbaciones alrededor del ELE crecen o se atenúan con el tiempo:

- Si todos los autovalores tienen parte real negativa, el equilibrio es estable.  
- Si alguno tiene parte real positiva, el equilibrio es inestable.


In [3]:
J_ELE = J.subs({S: N, I: 0})

eigs_ELE = J_ELE.eigenvals()
print("Autovalores simbólicos del ELE:")
display(eigs_ELE)


Autovalores simbólicos del ELE:


{N*beta - gamma - mu: 1, -mu: 1}

##

## Sustituir parámetros y calcular el Equilibrio Endémico

Se sustituyen los valores numéricos de los parámetros obtenidos.

In [4]:
params = {
    beta: 0.5 / 1000,
    gamma: 0.1,
    mu: 0.02,
    N: 1000
}

S_end = (params[gamma] + params[mu]) / params[beta]
I_end = params[mu] * (params[N] - S_end) / (params[gamma] + params[mu])
(S_end, I_end)


(240.0, 126.66666666666667)

## Jacobiano numérico en el Equilibrio Endémico

Se sustituye en el Jacobiano simbólico tanto los parámetros numéricos como las coordenadas del equilibrio endémico $(S^*, I^*)$  Esto produce una matriz 2x2, que describe el comportamiento local del sistema alrededor del punto endémico.


In [5]:
J_endemic = J.subs(params).subs({S: S_end, I: I_end})

J_num = np.array(J_endemic.tolist()).astype(np.float64)
print("Jacobiano numérico evaluado en el equilibrio endémico:")
print(J_num)


Jacobiano numérico evaluado en el equilibrio endémico:
[[-8.33333333e-02 -1.20000000e-01]
 [ 6.33333333e-02 -1.38777878e-17]]


## Autovalores numéricos del equilibrio endémico

Con la matriz Jacobiana numérica, se calculan los autovalores.
En la simulación, los autovalores presentan partes reales negativas,confirmando que el equilibrio endémico es estable.

In [6]:
eigvals_endemic = np.linalg.eigvals(J_num)
print("Autovalores numéricos del equilibrio endémico:")
print(eigvals_endemic)


Autovalores numéricos del equilibrio endémico:
[-0.04166667+0.07657603j -0.04166667-0.07657603j]


## Analisis

1. **ELE — segundo autovalor (forma simbólica).**  

   Muestre algebraicamente cómo el segundo autovalor del ELE es igual a  
   $$
   \lambda_2 \;=\; (\gamma+\mu)\,(R_0-1).
   $$  
   Luego, explique en palabras:  
   ¿Cómo confirma esto su hallazgo teórico de que la inestabilidad ocurre cuando $R_0>1$?

      Se parte del segundo autovalor simbólico en el ELE:
      $$
      \lambda_2 \;=\; N\beta - \gamma - \mu.
      $$

      Definimos
      $$
      R_0 \;=\; \frac{\beta N}{\gamma+\mu}.
      $$

      Entonces,
      $$
      \begin{aligned}
      (\gamma+\mu)(R_0-1)
      &= (\gamma+\mu)\!\left(\frac{\beta N}{\gamma+\mu}-1\right) \\
      &= (\gamma+\mu)\!\left(\frac{\beta N - (\gamma+\mu)}{\gamma+\mu}\right) \\
      &= \beta N - (\gamma+\mu) \\
      &= N\beta - \gamma - \mu \;=\; \lambda_2.
      \end{aligned}
      $$

      Como $\gamma+\mu>0$, si $R_0>1$ entonces $\lambda_2>0$: 
      
      el ELE es inestable con cualquier perturbación hacia $I>0$ crece.

2. **Autovalores numéricos en el equilibrio endémico.**  
   Examine los autovalores numéricos que calculó para el equilibrio endémico. ¿Sus partes reales son positivas o negativas? Con base en estas reglas, ¿el equilibrio endémico es estable o inestable?
   
      $$
      \lambda_{1,2}=-0.04166667 \pm 0.07657603\,i.
      $$
      Las partes reales son negativas $(-0.0417)$. Por lo tanto, el equilibrio endémico es un foco estable : la dinámica presenta oscilaciones amortiguadas que decaen hacia $(S^*,I^*)$.

3. **Consistencia entre estabilidad y trayectorias.**  
   Compare los resultados de su análisis de estabilidad (Parte 2) con el comportamiento visual de las trayectorias graficadas en la Parte 1. ¿Son consistentes los resultados? Explique cómo el hecho de que el ELE sea inestable y el endémico sea estable explica las trayectorias observadas.

   - En la Parte 1, ambas trayectorias convergen al equilibrio endémico y no al ELE.  
   - Esto concuerda con:
   - ELE inestable cuando $R_0>1$ (por $\lambda_2>0$).  
   - Endémico estable (partes reales negativas), lo que explica las curvas que giran  y se asientan en $(S^*,I^*)$.


4. **Efecto de mayor esperanza de vida ($\mu$ más pequeña) en el ELE.**  
   Si la esperanza de vida aumenta (disminuye $\mu$), ¿cómo afectaría esto al autovalor positivo del ELE cuando $R_0>1$? ¿Haría que el brote inicial fuera más rápido o más lento? Explique su razonamiento a partir de la fórmula del autovalor.

      Con $\lambda_2 = (\gamma+\mu)(R_0-1) = \beta N - (\gamma+\mu)$:
      - Si **disminuye $\mu$** (manteniendo $\beta,N,\gamma$ fijos), entonces $(\gamma+\mu)$ disminuye y
      $$
      \lambda_2 = \beta N - (\gamma+\mu) \ \text{aumenta}.
      $$
      - Además, $R_0=\frac{\beta N}{\gamma+\mu}$ aumenta.

      Entonces con $\mu$ más pequeña y $R_0>1$, el autovalor positivo se hace más grande, por lo que la divergencia del ELE es más rápida: el brote inicial progresa más rápido.


# Referencias

- Best, A. (2023, July 12). The SIR model with demographics. Introducing Mathematical Biology. https://sheffield.pressbooks.pub/introducingmathematicalbiology/chapter/the-sir-model-with-demographics/ 
- GeeksforGeeks. (2022, May 13). Overview of SIR epidemic model for Corona virus outbreak prediction. GeeksforGeeks. https://www.geeksforgeeks.org/machine-learning/overview-of-sir-epidemic-model-for-corona-virus-outbreak-prediction/ 
- GeeksforGeeks. (2022b, May 13). Overview of SIR epidemic model for Corona virus outbreak prediction. GeeksforGeeks. https://www.geeksforgeeks.org/machine-learning/overview-of-sir-epidemic-model-for-corona-virus-outbreak-prediction/
- Libretexts. (2022, July 18). 4.3: The SIR Epidemic Disease model. Mathematics LibreTexts. https://math.libretexts.org/Bookshelves/Applied_Mathematics/Mathematical_Biology_(Chasnov)/04%3A_Infectious_Disease_Modeling/4.03%3A_The_SIR_Epidemic_Disease_Model
